In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from sklearn import preprocessing
from sklearn.metrics import f1_score
from IPython.display import Image
import graphviz
import pydot
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import os
from dotenv import load_dotenv

load_dotenv() # load environment variables

Python-dotenv could not parse statement starting at line 8


True

In [36]:
df = pd.read_csv("/home/trucbui/PBLGit/random_forest/binary_classification_train.csv", sep="\t")
df = df.sort_values(['Label','Header'], ascending=(False,True))
print(df)

       Label      Header  Sequence length        Mass Most frequent aa  \
4929       1  A0A024RBG1              181   20403.285                E   
6943       1  A0A024SMV2              391   42741.476                A   
7617       1  A0A060S684              585   63631.507                A   
3466       1  A0A075TXZ3              561   61430.813                L   
3016       1  A0A077K8G3              407   45202.745                L   
...      ...         ...              ...         ...              ...   
16087      0      X2JAU8             1075  117164.578                E   
19427      0      X5HYT8              425   46670.153                A   
47371      0      X5LX76             1029  118226.052                S   
37737      0      X6R8D5              127   14504.179                S   
13501      0      X6R8R1              474   52161.620                L   

       Last aa is H  Last aa is Y  
4929              0             0  
6943              0             0  
761

In [3]:
'''
#Convert non numeric data into numeric data    
#TODO: find another way to convert. The numeric data should not be ranked
non_numeric_set=[df['Most frequent aa']]
le = preprocessing.LabelEncoder()
after_processing=[]
for s in non_numeric_set:
  after_processing.append(le.fit_transform(s))
df['Most frequent aa']=after_processing[0]

print(df)
'''


"\n#Convert non numeric data into numeric data    \n#TODO: find another way to convert. The numeric data should not be ranked\nnon_numeric_set=[df['Most frequent aa']]\nle = preprocessing.LabelEncoder()\nafter_processing=[]\nfor s in non_numeric_set:\n  after_processing.append(le.fit_transform(s))\ndf['Most frequent aa']=after_processing[0]\n\nprint(df)\n"

<font size="5">Working with Embeddings</font>

In [23]:
def read_esm2(path_to_esm2:str):
    
    with h5py.File(path_to_esm2) as hdf_handle:
        embeddings_max = []
        embeddings_min = []
        embeddings_average=[]
        embeddings_deviation=[]
    
        for header, emb in hdf_handle.items():
            emb = np.array(list(emb))
            embeddings_max.append(np.amax(emb))
            #embeddings_min.append(np.min(emb))


        return embeddings_max
    

embeddings_esm2_enzymes_30 = read_esm2(os.getenv("ESM2_ENZYMES_SPLIT_30"))
embeddings_esm2_non_enzymes = read_esm2(os.getenv("ESM2_NON_ENZYMES"))


In [24]:
emb=embeddings_esm2_enzymes_30+embeddings_esm2_non_enzymes
print(emb)

[12.089359, 10.99249, 11.732064, 11.502498, 12.043853, 12.276654, 12.542703, 12.737299, 11.778521, 12.01756, 11.590534, 11.794132, 14.102883, 11.473967, 12.427992, 13.630979, 12.267069, 12.887504, 11.342885, 11.051666, 11.244517, 10.991922, 11.263971, 12.557306, 11.573903, 12.218324, 10.926612, 11.329309, 12.313898, 11.633124, 10.813115, 12.352561, 12.007503, 12.485828, 12.2380705, 12.527522, 12.23991, 11.733987, 11.869087, 10.532462, 10.9344845, 11.13682, 10.666399, 13.168559, 12.433922, 12.101508, 11.620307, 11.072727, 12.153901, 11.777827, 12.097742, 12.336703, 10.8069, 11.898185, 10.783842, 12.34517, 11.951664, 12.385542, 10.180467, 10.84205, 12.322884, 11.67672, 12.241413, 12.865439, 11.230861, 12.69747, 11.539739, 11.613683, 12.500558, 10.194708, 12.095376, 11.042711, 12.577068, 11.919141, 12.770817, 13.102328, 12.075817, 12.778336, 11.282719, 12.735692, 11.36217, 13.71759, 11.78089, 12.561163, 11.482446, 11.782787, 13.1717, 10.491756, 12.042712, 12.101274, 12.381073, 12.78502, 1

In [37]:
df['Embeddings Max']=emb
print(df)

       Label      Header  Sequence length        Mass Most frequent aa  \
4929       1  A0A024RBG1              181   20403.285                E   
6943       1  A0A024SMV2              391   42741.476                A   
7617       1  A0A060S684              585   63631.507                A   
3466       1  A0A075TXZ3              561   61430.813                L   
3016       1  A0A077K8G3              407   45202.745                L   
...      ...         ...              ...         ...              ...   
16087      0      X2JAU8             1075  117164.578                E   
19427      0      X5HYT8              425   46670.153                A   
47371      0      X5LX76             1029  118226.052                S   
37737      0      X6R8D5              127   14504.179                S   
13501      0      X6R8R1              474   52161.620                L   

       Last aa is H  Last aa is Y  Embeddings Max  
4929              0             0       12.089359  
6943   

<font size="5">Train the model</font>

In [38]:
#Define the features and output:
y = np.array(df['Label'])
X = df.drop(['Label','Header','Sequence length','Most frequent aa'],axis=1)
#Split data into train an test, with test size of 20%:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
#Build the model:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred=rf.predict(X)

<font size="5">Evaluate the model</font>

In [39]:
print("accuracy on training set: %f" % rf.score(X_train, y_train))
print("accuracy on test set: %f" % rf.score(X_test, y_test))
f1 = f1_score(y, y_pred, zero_division="warn")
print('F1 score:', f1)

accuracy on training set: 0.999897
accuracy on test set: 0.782488
F1 score: 0.8806297441664324


<font size="5">Make a tree</font>

In [40]:
#Using only 5 nodes:
rf_5 = RandomForestClassifier(max_depth=5)
rf_5.fit(X_train, y_train)
# Pull out one tree from the forest:
tree = rf_5.estimators_[5]
# Export the image to a dot file:
export_graphviz(tree, out_file = 'tree.dot', feature_names = X.columns, rounded = True, precision = 1)
# Use dot file to create a graph:
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file:
graph.write_png('tree_rf.png')